In [ ]:
import torch

In [ ]:
!pip install torch_geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 11.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910459 sha256=4cba4f578b8e07c970d0aad7c21937813c751e7eaecc4bd168659cced9226f89
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch_geometric


In [ ]:
from torch_geometric.nn.attention import PerformerAttention

ModuleNotFoundError: ignored

In [ ]:
import torch_geometric

In [ ]:
from torch_geometric.datasets import Planetoid

In [ ]:
dataset=Planetoid(root='/tmp/Cora', name='Cora')

Processing...
Done!


In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

In [ ]:
class GCN(torch.nn.Module):
  def __init__(self):
      super().__init__()
      self.conv1=GCNConv(dataset.num_node_features, 16)
      self.conv2=GCNConv(16, dataset.num_classes)


  def forward(self, data):
       x,edge_index=data.x,data.edge_index

       x = self.conv1(x, edge_index)
       x = F.relu(x)
       x = F.dropout(x,training=self.training)
       x = self.conv2(x, edge_index)


       return F.log_softmax(x, dim=1)


In [ ]:
##Training

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss= F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [ ]:
model.eval()
pred=model(data).argmax(dim=1)
correct=(pred[data.test_mask]==data.y[data.test_mask]).sum()
acc=int(correct)/int(data.test_mask.sum())
print(f'Accuracy:{acc:4f}')

Accuracy:0.802000
